In [18]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
parser = argparse.ArgumentParser(description='Sequence Modeling - (Permuted) Sequential MNIST')
parser.add_argument('--batch_size', type=int, default=64, metavar='N',
                    help='batch size (default: 64)')
parser.add_argument('--cuda', action='store_false',
                    help='use CUDA (default: True)')
parser.add_argument('--dropout', type=float, default=0.05,
                    help='dropout applied to layers (default: 0.05)')
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=20,
                    help='upper epoch limit (default: 20)')
parser.add_argument('--ksize', type=int, default=7,
                    help='kernel size (default: 7)')
parser.add_argument('--levels', type=int, default=8,
                    help='# of levels (default: 8)')
parser.add_argument('--log-interval', type=int, default=100, metavar='N',
                    help='report interval (default: 100')
parser.add_argument('--lr', type=float, default=2e-3,
                    help='initial learning rate (default: 2e-3)')
parser.add_argument('--optim', type=str, default='Adam',
                    help='optimizer to use (default: Adam)')
parser.add_argument('--nhid', type=int, default=25,
                    help='number of hidden units per layer (default: 25)')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed (default: 1111)')
parser.add_argument('--permute', action='store_true',
                    help='use permuted MNIST (default: false)')
args = parser.parse_args()

torch.manual_seed(args.seed)

usage: ipykernel_launcher.py [-h] [--batch_size N] [--cuda]
                             [--dropout DROPOUT] [--clip CLIP]
                             [--epochs EPOCHS] [--ksize KSIZE]
                             [--levels LEVELS] [--log-interval N] [--lr LR]
                             [--optim OPTIM] [--nhid NHID] [--seed SEED]
                             [--permute]
ipykernel_launcher.py: error: unrecognized arguments: -f /Users/rysul/Library/Jupyter/runtime/kernel-4e1d6b14-5258-4b4b-814b-3788c2127a39.json


SystemExit: 2

/Users/rysul/opt/anaconda3/envs/torchEnv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [21]:
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:41].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[41] = np.where(df[41]=='normal', 'normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[41]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [22]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class NSLKDDDataset(Dataset):
    def __init__(self, filename):
        self.data = pd.read_csv(fileName, header = None)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :41].values)
        stringLower = string[0].lower()
        xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        xMid = np.array(xData)
        xArray = xMid.transpose() # should be now (39, 200)
        
        # prepate y data
        self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        yArray = self.data.iloc[idx, 41]
        
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), torch.from_numpy(yArray) # returns torch tensor of x and y

In [58]:
params = {'batch_size': 64, 'shuffle': True}
fileNameTrain = 'KDDTrain+.csv'
fileNameTest = 'KDDTest+.csv'
datasetTrain = NSLKDDDataset(fileNameTrain)
datasetTest = NSLKDDDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)


In [59]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 39
seq_length = int(200)
epochs = 100
steps = 0

In [60]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6#hidden nodes times levels 
kernel_size = 5
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.05)


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [61]:
print(train_loader)

In [62]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        # print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        loss1 = torch.nn.CrossEntropyLoss()
        #loss = F.nll_loss(output, target) # negative log likelihood
        loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss.item()/100, steps))
            train_loss = 0

In [67]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        return test_loss

In [68]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.009125	Steps: 161400
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.003341	Steps: 181400
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001680	Steps: 201400


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.



Test set: Average loss: 0.0470, Accuracy: 12881/10000 (129%)

Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.000764	Steps: 232000
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.000508	Steps: 252000
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.000359	Steps: 272000

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 3 [6400/60000 (11%)]	Loss: 0.000234	Steps: 302600
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.000180	Steps: 322600
Train Epoch: 3 [19200/60000 (32%)]	Loss: 0.000138	Steps: 342600

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 4 [6400/60000 (11%)]	Loss: 0.000091	Steps: 373200
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.000072	Steps: 393200
Train Epoch: 4 [19200/60000 (32%)]	Loss: 0.000060	Steps: 413200

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 5 [6400/60000 (11%)]	Loss: 0.000046	Steps: 443800
Train Epoch: 5 [12800/60000 (21%)]	Loss: 0.000039	Steps: 463800
Train Epoch: 5 [19200/60000 (32%)]	Loss: 0.00003

Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.000002	Steps: 2460600

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 34 [6400/60000 (11%)]	Loss: 0.000002	Steps: 2491200
Train Epoch: 34 [12800/60000 (21%)]	Loss: 0.000002	Steps: 2511200
Train Epoch: 34 [19200/60000 (32%)]	Loss: 0.000002	Steps: 2531200

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 35 [6400/60000 (11%)]	Loss: 0.000002	Steps: 2561800
Train Epoch: 35 [12800/60000 (21%)]	Loss: 0.000002	Steps: 2581800
Train Epoch: 35 [19200/60000 (32%)]	Loss: 0.000002	Steps: 2601800

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 36 [6400/60000 (11%)]	Loss: 0.000002	Steps: 2632400
Train Epoch: 36 [12800/60000 (21%)]	Loss: 0.000002	Steps: 2652400
Train Epoch: 36 [19200/60000 (32%)]	Loss: 0.000002	Steps: 2672400

Test set: Average loss: 0.0000, Accuracy: 22544/10000 (225%)

Train Epoch: 37 [6400/60000 (11%)]	Loss: 0.000002	Steps: 2703000
Train Epoch: 37 [12800/600

KeyboardInterrupt: 